In [1]:
import os
import sys
notebook_path = os.getcwd()  # Get the current working directory
project_root = os.path.dirname(notebook_path)  # Get the parent directory
print(project_root)
print(notebook_path)
sys.path.append(project_root)
os.chdir(project_root)
sys.path.append(project_root + '/src')
sys.path = list(set(sys.path))
print(sys.path)


/home/alice/git-repo/task-extraction
/home/alice/git-repo/task-extraction/scripts
['', '/home/alice/git-repo/task-extraction/src', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '/home/alice/git-repo/task-extraction/venv/lib/python3.10/site-packages', '/usr/lib/python310.zip', '/home/alice/git-repo/task-extraction']


In [2]:
import json
from src.lb_annotation_utils import Article
from src.llm_utils import system_prompts, estimate_token_cost
import re
from openai import OpenAI
client = OpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=os.getenv("OPENROUTER_API_KEY"))

4e-05


In [ ]:
# Define constants
MAP_FOLDER = "data/neurovault_labeled_papers/"
INDEX_FILE = MAP_FOLDER + "nv_labeled_tasks_pmcid_map.json"
TASK_FILE_PATTERN = MAP_FOLDER + "nv_labeled_tasks_{chunk_number}.json"

In [3]:
ann = json.load(open('data/labelbuddy_annotations/nv_collection/annotations.json'))

In [4]:
# Set up filter logic
# exclusion criteria
exclude_types = ["resting_state", "meta-analysis"]
# excluded articles
excluded_articles = []
# included articles
included_articles = []  
# extract methods
for i in range(len(ann)):
    article_obj  = Article(ann[i])
    if article_obj.type in exclude_types:
        excluded_articles.append({"pmcid": article_obj.pmcid, "exclusion_reason":{"type": article_obj.type}})
    elif article_obj.methods == "No explicit methods section":
        excluded_articles.append({"pmcid": article_obj.pmcid, "exclusion_reason":{"methods": "No explicit methods section"}})
    else:
        included_articles.append(article_obj)


No subheaders found in body section


In [5]:
excluded_articles

[{'pmcid': 5324609,
  'exclusion_reason': {'methods': 'No explicit methods section'}},
 {'pmcid': 5090046, 'exclusion_reason': {'type': 'resting_state'}},
 {'pmcid': 10634720, 'exclusion_reason': {'type': 'meta-analysis'}}]

In [6]:
included_articles

In [7]:
total_cost = 0
for i in range(len(included_articles)):
    article_obj  = included_articles[i]
    if article_obj.methods!="No explicit methods section":
        item = {"pmcid": article_obj.pmcid, "methods": article_obj.methods}
        total_cost += estimate_token_cost(article_obj.methods, "text-embedding-3-small")
    else:
        item = {"pmcid": article_obj.pmcid, "methods": None}
print(total_cost)


0.0024749999999999998


In [8]:
def extract_using_llm(text:str, client:OpenAI, call_message:callable):
    messages = call_message(text)
    response = client.chat.completions.create(
        model = 'openai/gpt-4o-mini',
        messages = messages,
        seed = 42, 
        response_format = {"type": "json_object"}
    )
    return response.choices[0].message.content
def extract_cognitive_task(text:str):
    messages = [
        {"role": "system", "content": f"{system_prompts['CognitiveTask']}"},
    {"role": "user", "content": f"{text}"},
]
    return messages
def extract_cognitive_task_description(text:str):
    messages = [
        {"role": "system", "content": f"{system_prompts['CognitiveTaskDescription']}"},
    {"role": "user", "content": f"{text}"},
]   
    return messages



In [9]:
for i in range(len(included_articles)):
    methods = included_articles[i].methods
    output = extract_using_llm(text = methods, client = client, call_message = extract_cognitive_task)
    if re.search(r'null', output):
        output_description = extract_using_llm(text = methods, client = client, call_message = extract_cognitive_task_description)
        included_articles[i].llm.cognitive_task_description = output_description
    else:
        included_articles[i].llm.cognitive_task = output


In [10]:
for i in range(len(included_articles)):
    print(included_articles[i].pmcid)
    print(included_articles[i].llm.cognitive_task)
    print(included_articles[i].llm.cognitive_task_description)
    print("\n")

8318202
{"cognitive_task": ["attentive listening", "word repetition"]}
None


9202476
{"cognitive_task": ["match-to-sample (MTS) task", "visual search", "visual attention", "reaction time measurement"]}
None




In [ ]:
def get_nv_labeled_tasks(pmcid, index_folder,  index_file,json_file_pattern):
    nv_labeled_tasks = []
    for i in range(len(included_articles)):
        output = {k: {'pmcid': k, 'task': task_info_dict, 'pmid': v['pmid'], 'doi': v['doi']}}
        nv_labeled_tasks.append(output)
    return nv_labeled_tasks

In [11]:
from src.data_utils import calculate_bert_score
calculator = BERTScoreCalculator()

calculator.calculate_bert_score(included_articles[0].cognitive_task, included_articles[1].cognitive_task)

/home/alice/git-repo/task-extraction/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[0.999999995, 0.5852819331885631]


ImportError: cannot import name 'calculate_bert_score' from 'src.data_utils' (/home/alice/git-repo/task-extraction/src/data_utils.py)